# Supervised Machine Learning to Predict Price

## Goals

In this Jupyter Notebook, we'll predict price using supervised machine learning algorithms for continuous outcomes. This notebook highlights:

1. Data cleaning 
2. Feature selection
3. Model evaluation and selection

## Data Loading and Cleaning

In [1]:
import pandas as pd
import numpy as np

from sklearn import linear_model # Linear, ridge, and LASSO regression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error 

In [2]:
data_path = '/Users/danielchen/Desktop/GitHub/tutorials/Supervised Machine Learning Continuous Outcome/Data/sample.csv'
data = pd.read_csv(data_path)

In [3]:
data.head()

,loc1,loc2,para1,dow,para2,para3,para4,price
0,0,01,1,Mon,662,3000.0,3.8,73.49
1,9,99,1,Thu,340,2760.0,9.2,300.00
2,0,04,0,Mon,16,2700.0,3.0,130.00
3,4,40,1,Mon,17,12320.0,6.4,365.00
4,5,50,1,Thu,610,2117.0,10.8,357.50


For this exercise, we'll need to ensure that all of our data is numeric.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   loc1    10000 non-null  object 
 1   loc2    10000 non-null  object 
 2   para1   10000 non-null  int64  
 3   dow     10000 non-null  object 
 4   para2   10000 non-null  int64  
 5   para3   10000 non-null  float64
 6   para4   10000 non-null  float64
 7   price   10000 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 625.1+ KB


It appears that the columns `loc1`, `loc2`, and `dow` (day of week) contain non-numeric values (or are string characters as in the `dow` column). 

In [5]:
# We'll only keep rows that have numeric values in the loc1 column
data = data[data['loc1'].str.contains('\d')]

# Convert loc1 and loc2 from strings to numeric
to_numeric_cols = ['loc' + str(num) for num in range(1, 3)]
data[to_numeric_cols] = data[to_numeric_cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Drop any NA values
data.dropna(inplace=True)


Let's check the number of observations that are left and the data types of the `loc` columns

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9993 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   loc1    9993 non-null   float64
 1   loc2    9993 non-null   float64
 2   para1   9993 non-null   int64  
 3   dow     9993 non-null   object 
 4   para2   9993 non-null   int64  
 5   para3   9993 non-null   float64
 6   para4   9993 non-null   float64
 7   price   9993 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 702.6+ KB


It looks like the `dow` column still contains strings. Let's convert the categories into numeric values.

In [7]:
dow_catgeories = data['dow'].unique().tolist()
dow_numeric = [x for x in range(1, len(dow_catgeories) + 1)]
replacement_dict = dict(zip(dow_catgeories, dow_numeric))

data['dow'] = data['dow'].replace(replacement_dict)

One final check on our data types:

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9993 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   loc1    9993 non-null   float64
 1   loc2    9993 non-null   float64
 2   para1   9993 non-null   int64  
 3   dow     9993 non-null   int64  
 4   para2   9993 non-null   int64  
 5   para3   9993 non-null   float64
 6   para4   9993 non-null   float64
 7   price   9993 non-null   float64
dtypes: float64(5), int64(3)
memory usage: 702.6 KB


# Identifying Features

For this walkthrough, we'll identify the best features based on their correlation to our outcome variable `price`. This isn't the only method for feature selection, though. LASSO regression, VIF score, and subset selection are also methods for identifying features. We justify using correlation here since our target variables should at least be related to our outcome variable.

In [9]:
correlations = (abs(data.corr()['price'])
                  .to_frame()
                  .sort_values(by='price', ascending=False)[1::])
correlations                 

,price
para2,0.551222
para4,0.517614
para3,0.356949
para1,0.074555
loc1,0.044079
loc2,0.043543
dow,0.001043


It looks like `para2`, `para4`, and `para3` are the most related to `price`, so we'll use those for our model.

In [10]:
features = correlations.nlargest(n=3, columns='price').index.tolist()

# Determining an Algorithm

Now that our data is in the right format (i.e., we've removed all non-numeric data and missing data), we need to identify the model the optimal algorithm to use. In this instance, by optimal, we are seeking to identify the model that makes the most accurate predictions, or the one that minimizes the mean squared error (the difference between the actual values and our predictions.)

In [11]:
# Create a list of tuples specifiying the model name and an instance of the model
algorithms = [
    ('Linear Regression', linear_model.LinearRegression()),
    ('Ridge', linear_model.Ridge()),
    ('LASSO', linear_model.Lasso()),
    ('Random Forest', RandomForestRegressor()),
    ('XGBoost', XGBRegressor())
]

# Create a list of measures to which we will evaluate the model's performance
measures = ['r2', 'neg_mean_squared_error']


Next, we'll determine our feature and target set, splitting our data into training and testing sets.

In [12]:
# Identify features and target
X = data[features].values
y = data['price'].values

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

We'll iterate over each algorithm, conduct k-fold cross validation on each estimator and create a dataframe summarizing the mean results for two popular evaluation scores: R-squared and mean squared error.

In [13]:
# Create an empty list to store the dataframe summarizing our results
test_harness_dfs = []

# Iterate over each algorithm, extract results into a dataframe, and append to 
# empty list
for name, algorithm in algorithms:
    results = cross_validate(
        estimator=algorithm,
        X=X_train,
        y=y_train,
        cv=10,
        scoring=measures
    )
    test_harness_df = pd.DataFrame({
        'algorithm': [name],
        'r2': [results['test_r2'].mean()],
        'mse': [results['test_neg_mean_squared_error'].mean() * -1]
    })
    test_harness_dfs.append(test_harness_df)

# Stack all individual dataframes
test_harness_results = pd.concat(test_harness_dfs)

It looks like our random forest performed the best in terms of minimizing the mean squared error:

In [14]:
test_harness_results.nsmallest(columns='mse', n=1)


,algorithm,r2,mse
0,Random Forest,0.630996,27367.18122


And it also looks like it performed best in terms of maximizing the R-squared:

In [15]:
test_harness_results.nlargest(columns='r2', n=1)

,algorithm,r2,mse
0,Random Forest,0.630996,27367.18122


# Tuning the Hyperparameters

Tuning hyperparemeters is vital for optimizing models. Now that we've determined that the random forest regressor is the optimal estimator, we'll need to tweak parameters such as the number of trees in our forest. 

Tweaking paramaters optimize the model by correcting for overfitting or underfitting. If we have too many trees, our model may learn our training data too well, and our model may not generalize well to new data. 


In [16]:
# Create a dictionary of values to randomly test
random_grid = {
    'n_estimators': [5, 20, 50, 100],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [_ for _ in np.linspace(start=10, stop=120, num=120)],
    'min_samples_split': [2, 6, 10],
    'min_samples_leaf': [1, 3, 4],
    'bootstrap': [True, False]
}

In [17]:
rf_random_grid = RandomizedSearchCV(
    estimator=RandomForestRegressor(), 
    param_distributions=random_grid,
    n_iter=100,
    cv=5,
    verbose=2,
    random_state=10,
    n_jobs=-1
)

rf_random_grid.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[CV] END bootstrap=True, max_depth=103.36134453781513, max_features=auto, min_samples_leaf=3, min_samples_split=10, n_estimators=20; total time=   0.3s
[CV] END bootstrap=True, max_depth=103.36134453781513, max_features=auto, min_samples_leaf=3, min_samples_split=10, n_estimators=20; total time=   0.3s
[CV] END bootstrap=True, max_depth=103.36134453781513, max_features=auto, min_samples_leaf=3, min_samples_split=10, n_estimators=20; total time=   0.3s
[CV] END bootstrap=True, max_depth=103.36134453781513, max_features=auto, min_samples_leaf=3, min_samples_split=10, n_estimators=20; total time=   0.3s
[CV] END bootstrap=True, max_depth=103.36134453781513, max_features=auto, min_samples_leaf=3, min_samples_split=10, n_estimators=20; total time=   0.4s
[CV] END bootstrap=False, max_depth=19.2436974789916, max_features=auto, min_samples_leaf=3, min_samples_split=2, n_estimators=5; total time=   0.1s
[CV] END bootstrap=False, max_depth=19.2436974789916, max_features=auto, min_samples_leaf=3

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10.0, 10.92436974789916,
                                                      11.84873949579832,
                                                      12.77310924369748,
                                                      13.69747899159664,
                                                      14.6218487394958,
                                                      15.546218487394958,
                                                      16.470588235294116,
                                                      17.39495798319328,
                                                      18.319327731092436,
                                                      19.2436974789916,
                                                      20.168067226890756,
                   

Let's identify the best parameters from our random grid search:

In [18]:
rf_random_grid.best_params_

{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 29.411764705882355,
 'bootstrap': True}

Next we fit create an instance of the `RandomForestRegressor` with the optimal hyperparameters.

In [19]:
tuned_rf = RandomForestRegressor(**rf_random_grid.best_params_)

In [20]:
# FIt the estimator
tuned_rf.fit(X_train, y_train)

# Make predictions using training and testing data
ytrain_predictions = tuned_rf.predict(X_train)
ytest_predictions = tuned_rf.predict(X_test)


# Evaluate R2 and MSE on the training and testing predictions
pred_summary = pd.DataFrame({
    'algorithm': ['Untuned RandomForest', 'Tuned RandomForest', 'Tuned RandomForest'],
    'split': ['test', 'train', 'test'],
    'r2': [
        test_harness_results.nlargest(columns='r2', n=1)['r2'].values[0],
        tuned_rf.score(X_train, y_train), 
        tuned_rf.score(X_test, y_test)
    ],
    'mse': [
        test_harness_results.nsmallest(columns='mse', n=1)['mse'].values[0],
         mean_squared_error(y_true=y_train, y_pred=ytrain_predictions), 
         mean_squared_error(y_true=y_test, y_pred=ytest_predictions)
    ]
})

pred_summary

,algorithm,split,r2,mse
0,Untuned RandomForest,test,0.630996,27367.181220
1,Tuned RandomForest,train,0.809002,14259.464247
2,Tuned RandomForest,test,0.700460,25219.633554


While it looks like the MSE from the tuned random forest on the testing set is still relatively high compared to the MSE From the tuned model on the training set, let's see what it would have looked like if we used all the default parameters from the `RandomForestRegressor()` class.

In [21]:
# Instantiate with default args and parameters
untuned_rf = RandomForestRegressor()

# Fit on the training data
untuned_rf.fit(X_train, y_train)

# Make predictions on the training and testing splits
ytrain_predictions_untuned = untuned_rf.predict(X_train)
ytest_predictions_untuned = untuned_rf.predict(X_test)

# Add untuned results to our summary dataframe from earlier
pred_summary.loc[len(pred_summary.index)] = [
    'Untuned RandomForest Refit',
    'train',
    untuned_rf.score(X_train, y_train),
    mean_squared_error(y_true=y_train, y_pred=ytrain_predictions_untuned)
]
pred_summary.loc[len(pred_summary.index)] = [
    'Untuned RandomForest Refit',
    'test',
    untuned_rf.score(X_test, y_test),
    mean_squared_error(y_true=y_test, y_pred=ytest_predictions_untuned)
]

# Re-label to make clear
pred_summary.iloc[0, 0] = 'Untuned RandomForest from Test Harness'

pred_summary

,algorithm,split,r2,mse
0,Untuned RandomForest from Test Harness,test,0.630996,27367.181220
1,Tuned RandomForest,train,0.809002,14259.464247
2,Tuned RandomForest,test,0.700460,25219.633554
3,Untuned RandomForest Refit,train,0.946112,4023.141005
4,Untuned RandomForest Refit,test,0.670748,27721.260637


Hypertuning the model has a fairly noticeable effect on reducing overfitting that occurs in rows 3 and 4 of the summary table. Without tuning the estimator, the random forest learns the training data very very well, resulting in low bias but high variance. The model is very sensitive to small changes in the data, which is evident in how the MSE from the untuned testing split is roughly 7 times as large as the MSE from the untuned training split.

We see that hypertuning has reduced gap between the MSE in the training versus testing split. The MSE in the hypertuned testing split is about 1.75 times as large as the MSE in the hypertuned training split. Although there's still room for improvement, the hypertuned model still outperforms the untuned model, as we've reduced the MSE by roughly 10% between the hypertuned/untuned testing splits.